In [1]:
import FinanceDataReader as fdr
import requests
import pandas as pd
import time
import ta
import openpyxl
import warnings
import cloudscraper
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')

import datetime
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

from defs_pred_auto import Data_Scrap_Pred
from datetime import datetime, timedelta
from pykrx import stock
from pykrx import bond
from PublicDataReader import Fred
from functools import reduce
from collections import OrderedDict
from defs_pred import Marcap, filter_df
from io import BytesIO
from openpyxl.utils.dataframe import dataframe_to_rows
from ics import Calendar, Event

In [9]:
import pandas as pd

# 데이터프레임 생성 예시
data = {
    'A': [1, 2, 3],
    'B': [4, 5, 6]
}
df = pd.DataFrame(data)

# 데이터프레임을 문자열로 변환
df_str = df.to_string()

print(df_str)

   A  B
0  1  4
1  2  5
2  3  6
